In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
import nltk as nl
from nltk import word_tokenize, pos_tag, bigrams
from nltk.parse.api import ParserI

sns.set
pd.options.display.max_colwidth = 200

In [2]:
data_raw = pd.read_csv('C:\\Users\\DELL\\Desktop\\Akshay\\Datasets\\factoid_questions_classification\\factoid_questions_dataset.txt', sep='delimiter', header=None, names=['Factoids'], dtype={'Factoids':str})

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data_raw

,Factoids
0,DESC:manner How did serfdom develop in and then leave Russia ?
1,ENTY:cremat What films featured the character Popeye Doyle ?
2,DESC:manner How can I find a list of celebrities ' real names ?
3,ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?
4,ABBR:exp What is the full form of .com ?
...,...
995,DESC:manner How do you look up criminal records on the Internet ?
996,"LOC:other Where is the actress , Marion Davies , buried ?"
997,DESC:desc What is an example of an actual case of using the Fourteenth Amendment ?
998,HUM:ind Who was the Charlie perfume woman ?


In [4]:
data_raw['Factoids'] = data_raw.Factoids.str.replace(':', ' ')
data_raw['Class'] = data_raw.Factoids.str.split('\s+').str[0]
data_raw['Sub-Class'] = data_raw.Factoids.str.split('\s+').str[1]
data_raw['Questions'] = data_raw.Factoids.str.split('\s+').str[2:]
data_raw.drop('Factoids', axis=1)

,Class,Sub-Class,Questions
0,DESC,manner,"[How, did, serfdom, develop, in, and, then, leave, Russia, ?]"
1,ENTY,cremat,"[What, films, featured, the, character, Popeye, Doyle, ?]"
2,DESC,manner,"[How, can, I, find, a, list, of, celebrities, ', real, names, ?]"
3,ENTY,animal,"[What, fowl, grabs, the, spotlight, after, the, Chinese, Year, of, the, Monkey, ?]"
4,ABBR,exp,"[What, is, the, full, form, of, .com, ?]"
...,...,...,...
995,DESC,manner,"[How, do, you, look, up, criminal, records, on, the, Internet, ?]"
996,LOC,other,"[Where, is, the, actress, ,, Marion, Davies, ,, buried, ?]"
997,DESC,desc,"[What, is, an, example, of, an, actual, case, of, using, the, Fourteenth, Amendment, ?]"
998,HUM,ind,"[Who, was, the, Charlie, perfume, woman, ?]"


In [5]:
questions = data_raw.Questions.to_numpy()
questions

array([list(['How', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'Russia', '?']),
       list(['What', 'films', 'featured', 'the', 'character', 'Popeye', 'Doyle', '?']),
       list(['How', 'can', 'I', 'find', 'a', 'list', 'of', 'celebrities', "'", 'real', 'names', '?']),
       list(['What', 'fowl', 'grabs', 'the', 'spotlight', 'after', 'the', 'Chinese', 'Year', 'of', 'the', 'Monkey', '?']),
       list(['What', 'is', 'the', 'full', 'form', 'of', '.com', '?']),
       list(['What', 'contemptible', 'scoundrel', 'stole', 'the', 'cork', 'from', 'my', 'lunch', '?']),
       list(['What', 'team', 'did', 'baseball', "'s", 'St.', 'Louis', 'Browns', 'become', '?']),
       list(['What', 'is', 'the', 'oldest', 'profession', '?']),
       list(['What', 'are', 'liver', 'enzymes', '?']),
       list(['Name', 'the', 'scar-faced', 'bounty', 'hunter', 'of', 'The', 'Old', 'West', '.']),
       list(['When', 'was', 'Ozzy', 'Osbourne', 'born', '?']),
       list(['Why', 'do', 'heavier', '

In [6]:
postagged = [pos_tag(question) for question in questions]
postagged

[[('How', 'WRB'),
  ('did', 'VBD'),
  ('serfdom', 'VB'),
  ('develop', 'VB'),
  ('in', 'IN'),
  ('and', 'CC'),
  ('then', 'RB'),
  ('leave', 'VB'),
  ('Russia', 'NNP'),
  ('?', '.')],
 [('What', 'WP'),
  ('films', 'VBD'),
  ('featured', 'VBD'),
  ('the', 'DT'),
  ('character', 'NN'),
  ('Popeye', 'NNP'),
  ('Doyle', 'NNP'),
  ('?', '.')],
 [('How', 'WRB'),
  ('can', 'MD'),
  ('I', 'PRP'),
  ('find', 'VB'),
  ('a', 'DT'),
  ('list', 'NN'),
  ('of', 'IN'),
  ('celebrities', 'NNS'),
  ("'", 'POS'),
  ('real', 'JJ'),
  ('names', 'NNS'),
  ('?', '.')],
 [('What', 'WP'),
  ('fowl', 'NN'),
  ('grabs', 'VBD'),
  ('the', 'DT'),
  ('spotlight', 'NN'),
  ('after', 'IN'),
  ('the', 'DT'),
  ('Chinese', 'JJ'),
  ('Year', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('Monkey', 'NNP'),
  ('?', '.')],
 [('What', 'WP'),
  ('is', 'VBZ'),
  ('the', 'DT'),
  ('full', 'JJ'),
  ('form', 'NN'),
  ('of', 'IN'),
  ('.com', 'NN'),
  ('?', '.')],
 [('What', 'WP'),
  ('contemptible', 'JJ'),
  ('scoundrel', 'NN'),
  

In [17]:
pos_tags = []

for i in range(len(postagged)):
    postagged[i]
    
    tags = []
    
    for j in range(len(postagged[i])):
        tags.append(postagged[i][j][1])
    
    pos_tags.append(tags)

pos_tags
pos_tags = np.array(pos_tags)
pos_tags

array([list(['WRB', 'VBD', 'VB', 'VB', 'IN', 'CC', 'RB', 'VB', 'NNP', '.']),
       list(['WP', 'VBD', 'VBD', 'DT', 'NN', 'NNP', 'NNP', '.']),
       list(['WRB', 'MD', 'PRP', 'VB', 'DT', 'NN', 'IN', 'NNS', 'POS', 'JJ', 'NNS', '.']),
       list(['WP', 'NN', 'VBD', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', '.']),
       list(['WP', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'NN', '.']),
       list(['WP', 'JJ', 'NN', 'VBD', 'DT', 'NN', 'IN', 'PRP$', 'NN', '.']),
       list(['WP', 'NN', 'VBD', 'NN', 'POS', 'NNP', 'NNP', 'NNP', 'VB', '.']),
       list(['WP', 'VBZ', 'DT', 'JJS', 'NN', '.']),
       list(['WP', 'VBP', 'JJ', 'NNS', '.']),
       list(['VB', 'DT', 'JJ', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NNP', '.']),
       list(['WRB', 'VBD', 'NNP', 'NNP', 'VBN', '.']),
       list(['WRB', 'VBP', 'JJR', 'NNS', 'VBP', 'VB', 'RBR', '.']),
       list(['WP', 'VBD', 'DT', 'NNP', 'IN', 'DT', 'NNS', '.']),
       list(['WP', 'VBD', 'NNP', '.']),
       list(['WP', 'VBZ', 'VBN', 'DT', 'JJS', 'NN',

In [8]:
data_raw['POS_Tags'] = np.array(pos_tags)
data_raw

,Factoids,Class,Sub-Class,Questions,POS_Tags
0,DESC manner How did serfdom develop in and then leave Russia ?,DESC,manner,"[How, did, serfdom, develop, in, and, then, leave, Russia, ?]","[WRB, VBD, VB, VB, IN, CC, RB, VB, NNP, .]"
1,ENTY cremat What films featured the character Popeye Doyle ?,ENTY,cremat,"[What, films, featured, the, character, Popeye, Doyle, ?]","[WP, VBD, VBD, DT, NN, NNP, NNP, .]"
2,DESC manner How can I find a list of celebrities ' real names ?,DESC,manner,"[How, can, I, find, a, list, of, celebrities, ', real, names, ?]","[WRB, MD, PRP, VB, DT, NN, IN, NNS, POS, JJ, NNS, .]"
3,ENTY animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?,ENTY,animal,"[What, fowl, grabs, the, spotlight, after, the, Chinese, Year, of, the, Monkey, ?]","[WP, NN, VBD, DT, NN, IN, DT, JJ, NN, IN, DT, NNP, .]"
4,ABBR exp What is the full form of .com ?,ABBR,exp,"[What, is, the, full, form, of, .com, ?]","[WP, VBZ, DT, JJ, NN, IN, NN, .]"
...,...,...,...,...,...
995,DESC manner How do you look up criminal records on the Internet ?,DESC,manner,"[How, do, you, look, up, criminal, records, on, the, Internet, ?]","[WRB, VB, PRP, VB, RP, JJ, NNS, IN, DT, NNP, .]"
996,"LOC other Where is the actress , Marion Davies , buried ?",LOC,other,"[Where, is, the, actress, ,, Marion, Davies, ,, buried, ?]","[WRB, VBZ, DT, NN, ,, NNP, NNP, ,, VBN, .]"
997,DESC desc What is an example of an actual case of using the Fourteenth Amendment ?,DESC,desc,"[What, is, an, example, of, an, actual, case, of, using, the, Fourteenth, Amendment, ?]","[WP, VBZ, DT, NN, IN, DT, JJ, NN, IN, VBG, DT, NNP, NNP, .]"
998,HUM ind Who was the Charlie perfume woman ?,HUM,ind,"[Who, was, the, Charlie, perfume, woman, ?]","[WP, VBD, DT, NNP, NN, NN, .]"


In [43]:
features = []
for i in range(len(questions)):
    postagged[i]
    
    question_features = np.array(())
    question_count = []
    token_count = []
    tokens = []
    postags = []
    
    for j in range(len(postagged[i])):
        question_count.append(i)
        token_count.append(j)
        tokens.append(postagged[i][j][0])
        postags.append(postagged[i][j][1])
    
    a = np.array(question_count)
    b = np.array(token_count)
    c = np.array(tokens)
    d = np.array(postags)
    
    question_features = np.column_stack((a,b,c,d))
    features.append(question_features)

features = np.array(features)
features

array([array([['0', '0', 'How', 'WRB'],
       ['0', '1', 'did', 'VBD'],
       ['0', '2', 'serfdom', 'VB'],
       ['0', '3', 'develop', 'VB'],
       ['0', '4', 'in', 'IN'],
       ['0', '5', 'and', 'CC'],
       ['0', '6', 'then', 'RB'],
       ['0', '7', 'leave', 'VB'],
       ['0', '8', 'Russia', 'NNP'],
       ['0', '9', '?', '.']], dtype='<U11'),
       array([['1', '0', 'What', 'WP'],
       ['1', '1', 'films', 'VBD'],
       ['1', '2', 'featured', 'VBD'],
       ['1', '3', 'the', 'DT'],
       ['1', '4', 'character', 'NN'],
       ['1', '5', 'Popeye', 'NNP'],
       ['1', '6', 'Doyle', 'NNP'],
       ['1', '7', '?', '.']], dtype='<U11'),
       array([['2', '0', 'How', 'WRB'],
       ['2', '1', 'can', 'MD'],
       ['2', '2', 'I', 'PRP'],
       ['2', '3', 'find', 'VB'],
       ['2', '4', 'a', 'DT'],
       ['2', '5', 'list', 'NN'],
       ['2', '6', 'of', 'IN'],
       ['2', '7', 'celebrities', 'NNS'],
       ['2', '8', "'", 'POS'],
       ['2', '9', 'real', 'JJ'],
       ['2

In [107]:
features[0]

array([['0', '0', 'How', 'WRB'],
       ['0', '1', 'did', 'VBD'],
       ['0', '2', 'serfdom', 'VB'],
       ['0', '3', 'develop', 'VB'],
       ['0', '4', 'in', 'IN'],
       ['0', '5', 'and', 'CC'],
       ['0', '6', 'then', 'RB'],
       ['0', '7', 'leave', 'VB'],
       ['0', '8', 'Russia', 'NNP'],
       ['0', '9', '?', '.']], dtype='<U11')

In [108]:
features[0].T

array([['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
       ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
       ['How', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave',
        'Russia', '?'],
       ['WRB', 'VBD', 'VB', 'VB', 'IN', 'CC', 'RB', 'VB', 'NNP', '.']],
      dtype='<U11')

In [120]:
features[1].T[3]

array(['WP', 'VBD', 'VBD', 'DT', 'NN', 'NNP', 'NNP', '.'], dtype='<U11')

In [123]:
a = pd.DataFrame(columns=list(range(len(features[0].T[3]))))
a.loc[0] = features[0].T[3]
a.loc[1] = features[0].T[3]
a

,0,1,2,3,4,5,6,7,8,9
0,WRB,VBD,VB,VB,IN,CC,RB,VB,NNP,.
1,WRB,VBD,VB,VB,IN,CC,RB,VB,NNP,.


In [39]:
postags = pd.DataFrame

for i in range(len(questions)):
    postagged[i]
    
    for j in range(len(postagged[i])):
        

pandas.core.frame.DataFrame

In [38]:
postags

pandas.core.frame.DataFrame

In [10]:
text = "How far is it from Denver to Aspen?"
tok_text = word_tokenize(text)
print(f"Tokens: {tok_text}")
bigrm_text = bigrams(tok_text)
print(f"Bigrams: {list(bigrm_text)}")
postag_text = pos_tag(tok_text)
print(f"Pos Tags: {list(postag_text)}")

Tokens: ['How', 'far', 'is', 'it', 'from', 'Denver', 'to', 'Aspen', '?']
Bigrams: [('How', 'far'), ('far', 'is'), ('is', 'it'), ('it', 'from'), ('from', 'Denver'), ('Denver', 'to'), ('to', 'Aspen'), ('Aspen', '?')]
Pos Tags: [('How', 'WRB'), ('far', 'RB'), ('is', 'VBZ'), ('it', 'PRP'), ('from', 'IN'), ('Denver', 'NNP'), ('to', 'TO'), ('Aspen', 'VB'), ('?', '.')]


In [11]:
range(len(postagged[1]))

range(0, 8)

In [12]:
a[1][1][3]

'VBD'

In [21]:
pos_tags[0][1]

'VBD'

In [81]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]